# Final Project: Dendrogram for Showing Group Distribution
- based on
  - https://plotly.com/python/tree-plots/
  - Not used
    - https://chart-studio.plotly.com/~pelicano_o/10/decision-tree-visualization/#plot
    - https://chart-studio.plotly.com/~pelicano_o/10/decision-tree-visualization/#/code

## Libraries

In [13]:
import igraph
from igraph import Graph, EdgeSeq
import pandas as pd
import plotly.graph_objects as go

In [3]:
# https://stackoverflow.com/questions/64849484/display-plotly-plot-inside-vs-code
import plotly.io as pio
pio.renderers.default = "notebook"

## Constants

In [17]:
# Get this figure: fig = py.get_figure("https://plotly.com/~pelicano_o/10/")
# Get this figure's data: data = py.get_figure("https://plotly.com/~pelicano_o/10/").data
# Add data to this figure: py.plot(Data([Scatter(x=[1, 2], y=[2, 3])]), filename ="plot from API (2)", fileopt="extend")


## Set Up Tree with igraph

In [24]:
nr_vertices = 25
v_label = list(map(str, range(nr_vertices)))
G = Graph.Tree(nr_vertices, 2) # 2 stands for children number
lay = G.layout('rt')

position = {k: lay[k] for k in range(nr_vertices)}
Y = [lay[k][1] for k in range(nr_vertices)]
M = max(Y)

es = EdgeSeq(G) # sequence of edges
E = [e.tuple for e in G.es] # list of edges

L = len(position)
# nodes
Xn = [position[k][0] for k in range(L)]
Yn = [2*M-position[k][1] for k in range(L)]
# edges
Xe = []
Ye = []
for edge in E:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
    Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]

# nodes of the labels

labels = v_label

Help on method _layout in module igraph.layout:

_layout(layout=None, *args, **kwds) method of igraph.Graph instance
    Returns the layout of the graph according to a layout algorithm.
    
    Parameters and keyword arguments not specified here are passed to the
    layout algorithm directly. See the documentation of the layout
    algorithms for the explanation of these parameters.
    
    Registered layout names understood by this method are:
    
      - C{auto}, C{automatic}: automatic layout
        (see L{Graph.layout_auto})
    
      - C{bipartite}: bipartite layout (see L{GraphBase.layout_bipartite})
    
      - C{circle}, C{circular}: circular layout
        (see L{GraphBase.layout_circle})
    
      - C{dh}, C{davidson_harel}: Davidson-Harel layout (see
        L{GraphBase.layout_davidson_harel})
    
      - C{drl}: DrL layout for large graphs (see L{GraphBase.layout_drl})
    
      - C{drl_3d}: 3D DrL layout for large graphs
        (see L{GraphBase.layout_drl})
    

In [21]:
df_nodes=pd.DataFrame({'Xn':Xn, 'Yn':Yn, 'labels':labels,'node_size':list(range(len(Xn)))})
df_edges=pd.DataFrame({'Xe':Xe, 'Ye':Ye,})

## Create Plotly Traces

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_edges["Xe"],
                   y=df_edges["Ye"],
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=df_nodes["Xn"],
                  y=df_nodes["Yn"],
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='square',
                                size=df_nodes["node_size"],
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))

## Create Text Inside the Circle via Annotations


In [5]:
def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
    L=len(pos)
    if len(text)!=L:
        raise ValueError('The lists pos and text must have the same len')
    annotations = []
    for k in range(L):
        annotations.append(
            dict(
                text=labels[k], # or replace labels with a different list for the text within the circle
                x=pos[k][0], y=2*M-position[k][1],
                xref='x1', yref='y1',
                font=dict(color=font_color, size=font_size),
                showarrow=False)
        )
    return annotations

## Add Axis Specifications and Create the Layout


In [7]:
axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(title= 'Tree with Reingold-Tilford Layout',
              annotations=make_annotations(position, v_label),
              font_size=12,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=40, r=40, b=85, t=100),
              hovermode='closest',
              plot_bgcolor='rgb(248,248,248)'
              )
fig.show()